# Add paper
Get papers by ORCID and create a new dataframe

In [ ]:
from orcid import download_author

In [ ]:
author = download_author("0000-0002-8428-1086")

In [ ]:
author.articles[1]

In [ ]:
author.save(f"authors/{author.orcid}.pickle")

In [ ]:
for article in author.articles:
    print(f"Title: {article.title}")
    print()
    print(article.abstract)
    print("-" * 80)

### Embed all works 

- input: title + abstract (if available)
- output: same embedding repr

Push output to Annoy